# Local Search  (Simulated Annealing)

### Setup and File paths

In [72]:
import math
import random
import csv
from collections import deque

# File paths
engine_file = "engines.txt"
tire_file = "tires.txt"
transmission_file = "transmissions.txt"
valid_cars_file = "valid_cars.csv"


### Car class

In [73]:
# Car class
target_path = []
valid_cars = set()

class Car:
    def __init__(self, engine, tire, transmission, roof):
        self.engine = engine
        self.tire = tire
        self.transmission = transmission
        self.roof = roof

    def __eq__(self, other):
        return (self.engine == other.engine and self.tire == other.tire and
                self.transmission == other.transmission and self.roof == other.roof)

    def __hash__(self):
        return hash((self.engine, self.tire, self.transmission, self.roof))

    def __repr__(self):
        return f"({self.engine}, {self.tire}, {self.transmission}, {self.roof})"

def load_cars(filename):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for engine, tire, transmission, roof in reader:
            valid_cars.add(Car(engine.strip(), tire.strip(), transmission.strip(), roof.strip()))

def content_reader(filename):
    with open(filename) as file:
        return [line.strip() for line in file]

def compare_with_target(car1, car2):
    mismatches = 0
    if car1.engine != car2.engine: mismatches += 1
    if car1.tire != car2.tire: mismatches += 1
    if car1.transmission != car2.transmission: mismatches += 1
    if car1.roof != car2.roof: mismatches += 1
    return mismatches

def delta_e(current, candidate, target):
    return compare_with_target(current, target) - compare_with_target(candidate, target)

def get_e(delta, level):
    t = 1.0 / level
    return math.exp(delta / t)

def simulate_component_changes(component_list, current_car, component_type, goal_car, seen, level, next_level_cars):
    global goal_reached
    for comp in component_list:
        if component_type == 'engine':
            candidate = Car(comp, current_car.tire, current_car.transmission, current_car.roof)
        elif component_type == 'tire':
            candidate = Car(current_car.engine, comp, current_car.transmission, current_car.roof)
        elif component_type == 'transmission':
            candidate = Car(current_car.engine, current_car.tire, comp, current_car.roof)
        elif component_type == 'roof':
            candidate = Car(current_car.engine, current_car.tire, current_car.transmission, comp)

        if candidate != current_car and candidate in valid_cars and candidate not in seen:
            dE = delta_e(current_car, candidate, goal_car)
            if dE > 0 or random.uniform(0, 1) <= get_e(dE, level):
                next_level_cars.append(candidate)
                seen.add(candidate)
                target_path.append(candidate)
                if candidate == goal_car:
                    return True
    return False

### Data Loading

In [74]:
# Load data

engines = content_reader(engine_file)
transmissions = content_reader(transmission_file)
tires = content_reader(tire_file)
roofs = ["Sunroof", "Moonroof", "Noroof", "Solid"]
load_cars(valid_cars_file)

# Define start and goal
start_car = Car("EFI", "Danlop", "AT", "Noroof")
goal_car = Car("EFI", "Danlop", "AT", "Solid")

if goal_car not in valid_cars:
    print("Goal car is not valid.")
    exit(1)

print("GOAL:", goal_car)
print("\nIs in valid_cars:", goal_car in valid_cars, "\n")

random.seed(42)

current_level_cars = deque()
seen = set()
seen.add(start_car)
current_level_cars.append(start_car)
target_path.append(start_car)

goal_reached = False
year = 0

while current_level_cars and not goal_reached:
    year += 1
    next_level_cars = deque()
    while current_level_cars and not goal_reached:
        current_car = current_level_cars.popleft()
        print(f"Year {year}: {current_car}")

        if simulate_component_changes(engines, current_car, 'engine', goal_car, seen, year, next_level_cars):
            goal_reached = True
            break
        if simulate_component_changes(tires, current_car, 'tire', goal_car, seen, year, next_level_cars):
            goal_reached = True
            break
        if simulate_component_changes(transmissions, current_car, 'transmission', goal_car, seen, year, next_level_cars):
            goal_reached = True
            break
        if simulate_component_changes(roofs, current_car, 'roof', goal_car, seen, year, next_level_cars):
            goal_reached = True
            break

    current_level_cars = next_level_cars

if goal_reached:
    print("\nGoal reached!")
    print("Path to goal:")
    for step in target_path:
        print(step)
    print(f"\nTotal years (steps) taken: {year+1}")
else:
    print("Goal not reached.")



GOAL: (EFI, Danlop, AT, Solid)

Is in valid_cars: True 

Year 1: (EFI, Danlop, AT, Noroof)

Goal reached!
Path to goal:
(EFI, Danlop, AT, Noroof)
(EFI, Goodyear, AT, Noroof)
(EFI, Cooper, AT, Noroof)
(EFI, Danlop, MT, Noroof)
(EFI, Danlop, AT, Solid)

Total years (steps) taken: 2
